### Elección del dataset
Para la tarea del modelo de regresión logística, elegí un dataset sobre la ciberguridad global que comprende los países más importantes del mundo. La elección de este datset se debe a que me pareció interesante poder aplicar el modelo para que pueda predecir que país tiene más probalidad de sufrir un ataque cibernético dependiendo del tipo de ataque sufrido.

### Importar dependencias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

### Impportar y crear el Df

In [69]:
df = pd.read_csv('Global_Cybersecurity_Threats_2015-2024.csv')

df.head()

,Country,Year,Attack Type,Target Industry,Financial Loss (in Million $),Number of Affected Users,Attack Source,Security Vulnerability Type,Defense Mechanism Used,Incident Resolution Time (in Hours)
0,China,2019,Phishing,Education,80.53,773169,Hacker Group,Unpatched Software,VPN,63
1,China,2019,Ransomware,Retail,62.19,295961,Hacker Group,Unpatched Software,Firewall,71
2,India,2017,Man-in-the-Middle,IT,38.65,605895,Hacker Group,Weak Passwords,VPN,20
3,UK,2024,Ransomware,Telecommunications,41.44,659320,Nation-state,Social Engineering,AI-based Detection,7
4,Germany,2018,Man-in-the-Middle,IT,74.41,810682,Insider,Social Engineering,VPN,68


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Country                              3000 non-null   object 
 1   Year                                 3000 non-null   int64  
 2   Attack Type                          3000 non-null   object 
 3   Target Industry                      3000 non-null   object 
 4   Financial Loss (in Million $)        3000 non-null   float64
 5   Number of Affected Users             3000 non-null   int64  
 6   Attack Source                        3000 non-null   object 
 7   Security Vulnerability Type          3000 non-null   object 
 8   Defense Mechanism Used               3000 non-null   object 
 9   Incident Resolution Time (in Hours)  3000 non-null   int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 234.5+ KB


In [33]:
df.isnull().sum()

Country                                0
Year                                   0
Attack Type                            0
Target Industry                        0
Financial Loss (in Million $)          0
Number of Affected Users               0
Attack Source                          0
Security Vulnerability Type            0
Defense Mechanism Used                 0
Incident Resolution Time (in Hours)    0
dtype: int64

In [70]:
df["Country"].unique()

array(['China', 'India', 'UK', 'Germany', 'France', 'Australia', 'Russia',
       'Brazil', 'Japan', 'USA'], dtype=object)

### Crear una columna para dividir los paises en occidente y oriente

In [72]:
oriente = ["China", "India", "Japan", "Russia", "Australia"]
occidente = ["USA", "UK", "Germany", "France", "Brazil"]

df["Region"] = df["Country"].apply(lambda x: "Oriente" if x in oriente else ("Occidente"))

df["Region"].value_counts()

Region
Occidente    1514
Oriente      1486
Name: count, dtype: int64

In [73]:
df["Region"] = df["Region"].map({"Oriente": 0, "Occidente": 1})

df.head()

,Country,Year,Attack Type,Target Industry,Financial Loss (in Million $),Number of Affected Users,Attack Source,Security Vulnerability Type,Defense Mechanism Used,Incident Resolution Time (in Hours),Region
0,China,2019,Phishing,Education,80.53,773169,Hacker Group,Unpatched Software,VPN,63,0
1,China,2019,Ransomware,Retail,62.19,295961,Hacker Group,Unpatched Software,Firewall,71,0
2,India,2017,Man-in-the-Middle,IT,38.65,605895,Hacker Group,Weak Passwords,VPN,20,0
3,UK,2024,Ransomware,Telecommunications,41.44,659320,Nation-state,Social Engineering,AI-based Detection,7,1
4,Germany,2018,Man-in-the-Middle,IT,74.41,810682,Insider,Social Engineering,VPN,68,1


### Vectorizar el tipo de ataque

In [75]:
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

In [78]:
x = vectorizer.fit_transform(df["Attack Type"])
y = df["Region"]

### Crear el modelo de regresión logística

In [79]:
model = LogisticRegression(max_iter=5000)

In [80]:
model.fit(x, y)

LogisticRegression(max_iter=5000)

### Predicción

In [52]:
df["Attack Type"].value_counts()

Attack Type
DDoS                 531
Phishing             529
SQL Injection        503
Ransomware           493
Malware              485
Man-in-the-Middle    459
Name: count, dtype: int64

In [100]:
attacks = ['DDos', 'Phishing', 'Ransomware', 'Malware', 'SQL Injection', 'Man-in-the-Middle']

for attack in attacks:
  attack_vector = vectorizer.transform([attack])
  result = model.predict(attack_vector)

  if result[0] == 1:
    print(f"El tipo de ataque {attack} es más común en Occidente.")
  else:
    print(f"El tipo de ataque {attack} es más común en Oriente.")

El tipo de ataque DDos es más común en Occidente.
El tipo de ataque Phishing es más común en Occidente.
El tipo de ataque Ransomware es más común en Oriente.
El tipo de ataque Malware es más común en Oriente.
El tipo de ataque SQL Injection es más común en Occidente.
El tipo de ataque Man-in-the-Middle es más común en Oriente.


In [103]:
for attack in attacks:
  probabilidad = model.predict_proba(vectorizer.transform([attack]))[0][1]
  print(f"La probabilidad de que el ataque {attack} sea más común en Occidente es {probabilidad}")

La probabilidad de que el ataque DDos sea más común en Occidente es 0.5063174534533663
La probabilidad de que el ataque Phishing sea más común en Occidente es 0.5332505673360362
La probabilidad de que el ataque Ransomware sea más común en Occidente es 0.4947194650591163
La probabilidad de que el ataque Malware sea más común en Occidente es 0.4906651388370337
La probabilidad de que el ataque SQL Injection sea más común en Occidente es 0.5185045481978602
La probabilidad de que el ataque Man-in-the-Middle sea más común en Occidente es 0.4801252546335011


### Guradar el modelo

In [106]:
joblib.dump(model, 'cybersecurity_model.pkl')

['cybersecurity_model.pkl']

In [119]:
clasificacion = joblib.load('cybersecurity_model.pkl')

attack_vector = vectorizer.transform([attacks][0])

y_predig = clasificacion.predict(attack_vector)
y_predig

array([1, 1, 0, 0, 1, 0])